In [75]:
import requests
import bs4
import re
import datetime as dt
import pandas as pd
import pdb

In [80]:
def print_news(term):
    """
    Prints the headlines from a google search of term. Sorts by most recent.
    """
    term = term.replace(' ', '+')
    url = 'https://www.google.com/search?q={}&source=lnms&tbm=nws'.format(term)

    r = requests.get(url)
    assert r.status_code == 200

    soup = bs4.BeautifulSoup(r.text, 'lxml')
    headlines = soup.find_all('div', {'class':'g'})

    headline_list = []
    source_list = []
    time_list = []

    for headline in headlines:
        link = headline.find('h3')
        source = headline.find('div', {'class':'slp'})

        headline = link.text
        source_time = source.text

        # Record headline
        headline_list.append(headline)

        # Record source
        source_text = re.search(r'^(.*) - ', source_time).group(1)
        source_list.append(source_text)

        # Convert dates
        time_string = re.search(r' - (.*)$', source_time).group(1)

        # Check if in format x days ago
        days_ago_format = re.search(r'^(\d+) (.*) ago$', time_string)

        if days_ago_format:
            if days_ago_format.group(2) == 'days':
                
                days_ago = int(days_ago_format.group(1))
                timestamp = dt.datetime.today() - dt.timedelta(days_ago)
            else:
                timestamp = dt.datetime.today()
        else:
            date_string = re.search(r'^\d{1,2} \D{3} \d{4}', time_string)
            if date_string:
                timestamp = dt.datetime.strptime(date_string.group(), '%d %b %Y')
            else:
                timestamp = None

        time_list.append(timestamp)

    news_list = zip(time_list, headline_list, source_list)
    for time, headline, source in sorted(news_list, reverse=True):
        print(time.date())
        print(headline)
        print(source)
        print()

In [82]:
term = 'Damian Lillard'

print_news(term)

2019-01-08
Damian Lillard on Blazers' upcoming schedule, 'I like where we are'
Blazer's Edge

2019-01-06
Trail Blazers' Damian Lillard: Leads team in scoring with 23 points
CBSSports.com

2019-01-06
Don't Fret Over Damian Lillard's Early NBA All-Star Game Returns
Blazer's Edge

2019-01-04
NBA All-Star voting: Trail Blazers' Damian Lillard 6th among ...
OregonLive.com

2019-01-04
Damian Lillard Sits No. 6 in First NBA All-Star Returns
Blazer's Edge

2019-01-03
Trail Blazers' Damian Lillard: Scores team-high 25 points in win
CBSSports.com

2019-01-03
Eggers Shares Two Touching Damian Lillard Stories
Blazer's Edge

2018-12-28
Oakland's Damian Lillard comes up clutch for Portland's last game at ...
ESPN

2018-12-18
Trail Blazers' Damian Lillard playing 'stronger ... more efficient'
USA TODAY

2018-12-11
Portland Trail Blazers: Has Damian Lillard surpassed Chris Paul?
Rip City Project

